<b><h4>Capstone Project - The Battle of Neighborhoods</h4></b>
</br> Week 1:
</br> <b>Introduction/Business Problem</b>

On 2010 an investor is studying the possibility of opening three TACOBELL (Fast Food of Mexican food) franchises in three different Brazilian states, however he is having trouble deciding which ones these will be, given that Brazil has 27 states.

Our job will be to assist this investor in identifying which states can bring the best return on their investment. We will be based on the number of Mexican emigrants and tourists in each state and assign a weight to each type of information, for example, a Mexican emigrant has a greater weight in the decision than a Mexican tourist.

<b>Data/Problem</b>

We will extract the information from Mexican migrants and tourists in Brazil based on the Brazilian government website. The analysis will be done with data based on 2010.
</br>
</br> <u>Reference</u>
</br>Emigrants: https://sidra.ibge.gov.br/home/ipca15/brasil
</br>Tourists: http://dadosefatos.turismo.gov.br/2016-02-04-11-53-05.html
</br>
</br>For each tourist weight 1 will be assigned. For each emigrant weight 3 will be assigned. In other words an emigrant is worth three times more than a tourist in our analysis.
</br>
</br>After that we will be able to have a vision by state of which region has greater influence from the Mexican community in Brazil, impacting directly in the decision of our investor.